In [2]:
!pip -q install rpy2
%load_ext rpy2.ipython


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
%%R
options(repos = c(CRAN = "https://cloud.r-project.org"))

if (!require("pacman")) install.packages("pacman")
pacman::p_load(tidyverse, ggplot2, dplyr, lubridate, stringr, readxl, data.table, gdata, scales, data.table)

source("functions.R")
monthlist <- sprintf("%02d", 1:2)
y <- 2014

* installing *source* package ‘remotes’ ...
** package ‘remotes’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (remotes)
* installing *source* package ‘pacman’ ...
** package ‘pacman’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path

Loading required package: pacman
Installing package into ‘/home/salizad/R/x86_64-conda-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)
also installing the dependency ‘remotes’

trying URL 'https://cloud.r-project.org/src/contrib/remotes_2.5.0.tar.gz'
Content type 'application/x-gzip' length 164496 bytes (160 KB)
downloaded 160 KB

trying URL 'https://cloud.r-project.org/src/contrib/pacman_0.5.1.tar.gz'
Content type 'application/x-gzip' length 274400 bytes (267 KB)
downloaded 267 KB


The downloaded source packages are in
	‘/tmp/RtmpXFS1hv/downloaded_packages’
Error in file(filename, "r", encoding = encoding) : 
  cannot open the connection
In addition: Warning messages:
1: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
  there is no package called ‘pacman’
2: In file(filename, "r", encoding = encoding) :
  cannot open file 'functions.R': No such file or directory
Error in file(filename, "r", encoding = encoding) : 
  cannot open the connection


RInterpreterError: Failed to parse and evaluate line 'options(repos = c(CRAN = "https://cloud.r-project.org"))\n\nif (!require("pacman")) install.packages("pacman")\npacman::p_load(tidyverse, ggplot2, dplyr, lubridate, stringr, readxl, data.table, gdata, scales, data.table)\n\nsource("functions.R")\nmonthlist <- sprintf("%02d", 1:2)\ny <- 2014\n'.
R error message: 'Error in file(filename, "r", encoding = encoding) : \n  cannot open the connection'
R stdout:
Loading required package: pacman
Installing package into ‘/home/salizad/R/x86_64-conda-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)
also installing the dependency ‘remotes’

trying URL 'https://cloud.r-project.org/src/contrib/remotes_2.5.0.tar.gz'
Content type 'application/x-gzip' length 164496 bytes (160 KB)
==================================================
downloaded 160 KB

trying URL 'https://cloud.r-project.org/src/contrib/pacman_0.5.1.tar.gz'
Content type 'application/x-gzip' length 274400 bytes (267 KB)
=================

# Plan (enrollment and contract) data


In [4]:
%%R

plan.data <- map_dfr(monthlist, ~ load_month(.x, y)) %>%
  arrange(contractid, planid, state, county, month) %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%                
  ungroup() %>%
  group_by(contractid, planid) %>%
  fill(plan_type, partd, snp, eghp, plan_name, .direction = "downup") %>%
  ungroup() %>%
  group_by(contractid) %>%
  fill(org_type, org_name, org_marketing_name, parent_org, .direction = "downup") %>%
  ungroup()

plan.data.dt <- as.data.table(plan.data)
setorder(plan.data.dt, contractid, planid, fips, year, month)
  
plan.year <- plan.data.dt[
, {
    nonmiss <- !is.na(enrollment)
    n <- sum(nonmiss)
    list(
      n_nonmiss = n,
      avg_enrollment = if (n>0) mean(enrollment[nonmiss]) else NA_real_,
      sd_enrollment  = if (n>1) sd(enrollment[nonmiss]) else NA_real_,
      min_enrollment = if (n>0) min(enrollment[nonmiss]) else NA_real_,
      max_enrollment = if (n>0) max(enrollment[nonmiss]) else NA_real_,
      first_enrollment = if (n>0) enrollment[which(nonmiss)[1]] else NA_real_,
      last_enrollment  = if (n>0) enrollment[tail(which(nonmiss), 1)] else NA_real_,
      state  = tail(state, 1),
      county = tail(county, 1),
      org_type = tail(org_type, 1),
      plan_type = tail(plan_type, 1),
      partd = tail(partd, 1),
      snp   = tail(snp, 1),
      eghp  = tail(eghp, 1),
      org_name = tail(org_name, 1),
      org_marketing_name = tail(org_marketing_name, 1),
      plan_name = tail(plan_name, 1),
      parent_org = tail(parent_org, 1),
      contract_date = tail(contract_date, 1)
    )
  },
by = .(contractid, planid, fips, year)
]

plan.data.2014 <- as_tibble(plan.year)

Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : 
  object 'monthlist' not found


RInterpreterError: Failed to parse and evaluate line '\nplan.data <- map_dfr(monthlist, ~ load_month(.x, y)) %>%\n  arrange(contractid, planid, state, county, month) %>%\n  group_by(state, county) %>%\n  fill(fips, .direction = "downup") %>%                \n  ungroup() %>%\n  group_by(contractid, planid) %>%\n  fill(plan_type, partd, snp, eghp, plan_name, .direction = "downup") %>%\n  ungroup() %>%\n  group_by(contractid) %>%\n  fill(org_type, org_name, org_marketing_name, parent_org, .direction = "downup") %>%\n  ungroup()\n\nplan.data.dt <- as.data.table(plan.data)\nsetorder(plan.data.dt, contractid, planid, fips, year, month)\n\nplan.year <- plan.data.dt[\n, {\n    nonmiss <- !is.na(enrollment)\n    n <- sum(nonmiss)\n    list(\n      n_nonmiss = n,\n      avg_enrollment = if (n>0) mean(enrollment[nonmiss]) else NA_real_,\n      sd_enrollment  = if (n>1) sd(enrollment[nonmiss]) else NA_real_,\n      min_enrollment = if (n>0) min(enrollment[nonmiss]) else NA_real_,\n      max_enroll

# Service area data


In [5]:
%%R
service.year <- map_dfr(monthlist, ~ load_month_sa(.x, y))

# Ensure stable order before fills
service.year <- service.year %>%
  arrange(contractid, fips, state, county, month)

# Fill missing identifiers/labels
service.year <- service.year %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%
  ungroup() %>%
  group_by(contractid) %>%
  fill(plan_type, partial, eghp, org_type, org_name, .direction = "downup") %>%
  ungroup()

# Collapse to yearly: one row per contract × county (fips) × year --------
service.data.2014 <- service.year %>%
  group_by(contractid, fips, year) %>%
  arrange(month, .by_group = TRUE) %>%
  summarize(
    state     = last(state),
    county    = last(county),
    org_name  = last(org_name),
    org_type  = last(org_type),
    plan_type = last(plan_type),
    partial   = last(partial),
    eghp      = last(eghp),
    ssa       = last(ssa),
    notes     = last(notes),
    .groups = "drop"
  )


In addition: Warning messages:
1: One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat) 
2: One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat) 


# Plan characteristics (premium) data


In [6]:
%%R
# Import data -------------------------------------------------------------

ma.path.a <- paste0("../../ma-data/ma/landscape/Extracted Data/2014LandscapeSource file MA_AtoM 05292014.csv")
ma.data.a <- read_csv(ma.path.a,
                      skip=6,
                      col_names=c("state","county","org_name","plan_name","plan_type","premium","partd_deductible",
                                  "drug_type","gap_coverage","drug_type_detail","contractid",
                                  "planid","segmentid","moop","star_rating"),
                      col_types = cols(
                        state = col_character(),
                        county = col_character(),
                        org_name = col_character(),
                        plan_name = col_character(),
                        plan_type = col_character(),
                        premium = col_character(),
                        partd_deductible = col_character(),
                        drug_type = col_character(),
                        gap_coverage = col_character(),
                        drug_type_detail = col_character(),
                        contractid = col_character(),
                        planid = col_double(),
                        segmentid = col_double(),
                        moop = col_character(),
                        star_rating = col_character()
                      )) %>%
  mutate_at(c('premium','partd_deductible'), ~str_replace(.,"-","0")) %>%
  mutate_at(c('premium','partd_deductible'), ~parse_number(.))



ma.path.b <- paste0("../../ma-data/ma/landscape/Extracted Data/2014LandscapeSource file MA_NtoW 05292014.csv")
ma.data.b <- read_csv(ma.path.b,
                      skip=6,
                      col_names=c("state","county","org_name","plan_name","plan_type","premium","partd_deductible",
                                  "drug_type","gap_coverage","drug_type_detail","contractid",
                                  "planid","segmentid","moop","star_rating"),
                      col_types = cols(
                        state = col_character(),
                        county = col_character(),
                        org_name = col_character(),
                        plan_name = col_character(),
                        plan_type = col_character(),
                        premium = col_character(),
                        partd_deductible = col_character(),
                        drug_type = col_character(),
                        gap_coverage = col_character(),
                        drug_type_detail = col_character(),
                        contractid = col_character(),
                        planid = col_double(),
                        segmentid = col_double(),
                        moop = col_character(),
                        star_rating = col_character()
                      )) %>%
  mutate_at(c('premium','partd_deductible'), ~str_replace(.,"-","0")) %>%
  mutate_at(c('premium','partd_deductible'), ~parse_number(.))


ma.data <- rbind(ma.data.a,ma.data.b)


mapd.path.a <- paste0("../../ma-data/ma/landscape/Extracted Data/PartCD/2014/Medicare Part D 2014 Plan Report 05292014.xls")
mapd.data.a <- read_xls(mapd.path.a,
                        range="A5:AA15859",
                        sheet="Alabama to Montana",
                        col_names=c("state","county","org_name","plan_name","contractid","planid","segmentid",
                                    "org_type","plan_type","snp","snp_type","benefit_type","below_benchmark",
                                    "national_pdp","premium_partc",
                                    "premium_partd_basic","premium_partd_supp","premium_partd_total",
                                    "partd_assist_full","partd_assist_75","partd_assist_50","partd_assist_25",
                                    "partd_deductible","deductible_exclusions","increase_coverage_limit",
                                    "gap_coverage","gap_coverage_type"))



mapd.path.b <- paste0("../../ma-data/ma/landscape/Extracted Data/PartCD/2014/Medicare Part D 2014 Plan Report 05292014.xls")
mapd.data.b <- read_xls(mapd.path.b,
                        range="A5:AA20305",
                        sheet="Nebraska to Wyoming",
                        col_names=c("state","county","org_name","plan_name","contractid","planid","segmentid",
                                    "org_type","plan_type","snp","snp_type","benefit_type","below_benchmark",
                                    "national_pdp","premium_partc",
                                    "premium_partd_basic","premium_partd_supp","premium_partd_total",
                                    "partd_assist_full","partd_assist_75","partd_assist_50","partd_assist_25",
                                    "partd_deductible","deductible_exclusions","increase_coverage_limit",
                                    "gap_coverage","gap_coverage_type"))
mapd.data <- rbind(mapd.data.a,mapd.data.b)

landscape.2014 <- mapd.clean.merge(ma.data=ma.data, mapd.data=mapd.data, y)

# Penetration data


In [8]:
%%R
ma.penetration <- map_dfr(monthlist, ~ load_month_pen(.x, y)) %>%
  arrange(state, county, month) %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%
  ungroup()

# Collapse to yearly (safe summaries; avoid NaN/Inf) ----------------------
pen.2014 <- ma.penetration %>%
  group_by(fips, state, county, year) %>%
  arrange(month, .by_group = TRUE) %>%
  summarize(
    n_elig  = sum(!is.na(eligibles)),
    n_enrol = sum(!is.na(enrolled)),

    avg_eligibles   = ifelse(n_elig  > 0, mean(eligibles, na.rm = TRUE), NA_real_),
    sd_eligibles    = ifelse(n_elig  > 1,  sd(eligibles,  na.rm = TRUE), NA_real_),
    min_eligibles   = ifelse(n_elig  > 0, min(eligibles,  na.rm = TRUE), NA_real_),
    max_eligibles   = ifelse(n_elig  > 0, max(eligibles,  na.rm = TRUE), NA_real_),
    first_eligibles = ifelse(n_elig  > 0, first(na.omit(eligibles)),     NA_real_),
    last_eligibles  = ifelse(n_elig  > 0,  last(na.omit(eligibles)),     NA_real_),

    avg_enrolled    = ifelse(n_enrol > 0, mean(enrolled,   na.rm = TRUE), NA_real_),
    sd_enrolled     = ifelse(n_enrol > 1,  sd(enrolled,    na.rm = TRUE), NA_real_),
    min_enrolled    = ifelse(n_enrol > 0, min(enrolled,    na.rm = TRUE), NA_real_),
    max_enrolled    = ifelse(n_enrol > 0, max(enrolled,    na.rm = TRUE), NA_real_),
    first_enrolled  = ifelse(n_enrol > 0, first(na.omit(enrolled)),       NA_real_),
    last_enrolled   = ifelse(n_enrol > 0,  last(na.omit(enrolled)),       NA_real_),

    ssa = last(ssa),
    .groups = "drop"
  )



# Rebate data


In [9]:
%%R

# Import data -------------------------------------------------------------

ma.path.a <- "../../ma-data/ma/cms-payment/2014/2014PartCPlan Level.xlsx"
risk.rebate.a <- read_xlsx(ma.path.a,range="A4:G2828",
                        col_names=c("contractid","planid","contract_name","plan_type",
                                    "riskscore_partc","payment_partc","rebate_partc"))
ma.path.b <- "../../ma-data/ma/cms-payment/2014/2014PartDPlans.xlsx"
risk.rebate.b <- read_xlsx(ma.path.b,range="A4:H3902",
                        col_names=c("contractid","planid","contract_name","plan_type",
                                    "directsubsidy_partd","riskscore_partd","reinsurance_partd",
                                    "costsharing_partd"))


risk.rebate.a <- risk.rebate.a %>%
  mutate(
    across(c(riskscore_partc, payment_partc, rebate_partc),
           ~ parse_number(as.character(.))),
    planid = as.numeric(planid),
    year   = 2014
  ) %>%
  select(contractid, planid, contract_name, plan_type,
         riskscore_partc, payment_partc, rebate_partc, year)    

risk.rebate.b <- risk.rebate.b %>%
  mutate(
    across(c(directsubsidy_partd, reinsurance_partd, costsharing_partd),
           ~ parse_number(as.character(.))),
    payment_partd = directsubsidy_partd + reinsurance_partd + costsharing_partd,
    planid        = as.numeric(planid)
  ) %>%
  select(contractid, planid, payment_partd,
         directsubsidy_partd, reinsurance_partd, costsharing_partd,
         riskscore_partd)  
  
rebate.2014 <- risk.rebate.a %>%
  left_join(risk.rebate.b, by=c("contractid","planid"))

# FFS costs data


In [ ]:
%%R

# Import data -------------------------------------------------------------

ffs.data <- read_csv("../../ma-data/ffs-costs/Extracted Data/Aged Only/aged14.csv",
                  skip=2,
                  col_names=FALSE, na=c("*","."))

ffs.data <- ffs.data[,1:15]
names(ffs.data) <- c("ssa","state","county_name","parta_enroll",
                    "parta_reimb","parta_percap","parta_reimb_unadj",
                    "parta_percap_unadj","parta_ime","parta_dsh",
                    "parta_gme","partb_enroll",
                    "partb_reimb","partb_percap",
                    "mean_risk")  

ffscosts.2014 <- ffs.data %>%
  select(ssa,state,county_name,parta_enroll,parta_reimb,
         partb_enroll,partb_reimb,mean_risk) %>%
  mutate(year=2014,
         ssa=as.numeric(ssa)) %>%
  mutate(across(c(parta_enroll, parta_reimb, partb_enroll, partb_reimb, mean_risk),
              ~ parse_number(as.character(.))))


# Merge data

In [10]:
%%R
ma.2014 <- plan.data.2014 %>%
  inner_join(service.data.2014 %>% select(contractid, fips),
             by = c("contractid","fips")) %>%
  filter(!state %in% c("VI","PR","MP","GU","AS",""),
         snp == "No",
         (planid < 800 | planid >= 900),
         !is.na(planid), !is.na(fips)) %>%
  left_join(pen.2014 %>% ungroup() %>% rename(state_long = state, county_long = county) %>%
            mutate(state_long=str_to_lower(state_long)) %>% 
            group_by(fips) %>% mutate(ncount=n()) %>% filter(ncount==1),
            by = c("fips"))

state.2014 <- ma.2014 %>%
  group_by(state) %>%
  summarize(state_name = last(state_long[!is.na(state_long)]), .groups = "drop")

full.2014 <- ma.2014 %>%
  left_join(state.2014, by = "state") %>%
  left_join(landscape.2014 %>% mutate(state=str_to_lower(state)), by = c("contractid","planid","state_name" = "state","county")) %>%
  left_join(rebate.2014 %>% select(-contract_name, -plan_type), by = c("contractid","planid")) %>%
  mutate(
    basic_premium = case_when(
      rebate_partc > 0 ~ 0,
      partd == "No" & !is.na(premium) & is.na(premium_partc) ~ premium,
      TRUE ~ premium_partc
    ),
    bid = case_when(
      rebate_partc == 0 & basic_premium > 0 ~ (payment_partc + basic_premium) / riskscore_partc,
      rebate_partc > 0  | basic_premium == 0 ~  payment_partc / riskscore_partc,
      TRUE ~ NA_real_
    )
  )

#  left_join(ffscosts.2014 %>% select(-state) %>% filter(!is.na(ssa)), by = c("ssa","year")) %>%
#  mutate(
#    avg_ffscost = case_when(
#      parta_enroll == 0 & partb_enroll == 0 ~ 0,
#      parta_enroll == 0 & partb_enroll >  0 ~ partb_reimb / partb_enroll,
#      parta_enroll >  0 & partb_enroll == 0 ~ parta_reimb / parta_enroll,
#      parta_enroll >  0 & partb_enroll >  0 ~ (parta_reimb / parta_enroll) + (partb_reimb / partb_enroll),
#      TRUE ~ NA_real_
#    )
#  )



In [14]:
%%R
write_csv(full.2014,"../data/output/data-2014.csv")